In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
##########################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
csv_prefix = 'scnn1a_all_slices.csv'
csv_columns = ['Date',
             'Slice/Loc/Rec',
             'Stim Location',
             'L4 barrel 1 Latency',
             'L4 barrel 2 Latency',
             'L4 barrel 3 Latency',
             'L4 barrel 1 SNR',
             'L4 barrel 2 SNR',
             'L4 barrel 3 SNR',
             'ROI jpg',
              'Age',
              'Infusion',
              'Sex',
              'Stim (uA)',
              'Delta latency 1 (barrel 1 - barrel 2)',
              'Delta latency 2 (barrel 3 - barrel 2)']

enable_photoZ_interaction = True
initialize_photoZ = False
snr_cutoff_signal = 5.0

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,ROI jpg,Age,Infusion,Sex,Stim (uA),Delta latency 1 (barrel 1 - barrel 2),Delta latency 2 (barrel 3 - barrel 2)
0,2/28/2023,02_01_01,L5,50.3009,50.8406,49.3457,2.94039,10.63090,6.48439,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-1.4949
1,2/28/2023,02_01_03,L5,50.6451,50.8291,49.8348,4.43717,9.88073,5.89912,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-0.9943
2,2/28/2023,02_02_02,L5,49.9882,51.3212,51.1805,2.04521,5.14209,2.92133,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,NaN
3,2/28/2023,03_01_01,L2/3,52.5719,51.2136,52.9013,9.40509,10.23720,5.78093,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.3583,1.6877
4,2/28/2023,03_02_01,L2/3,51.6168,50.1283,50.9250,25.20450,27.82770,26.27210,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.4885,0.7967
5,2/28/2023,03_02_02,L2/3,51.8399,50.0310,50.7536,14.73820,35.40020,21.88320,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.8089,0.7226
6,2/28/2023,03_02_12,L5,51.4097,50.5164,51.3004,7.26358,14.01600,14.95140,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,0.8933,0.7840
7,2/28/2023,06_01_01,L5,51.1367,51.0128,50.4326,5.72647,13.05830,13.06350,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,0.1239,-0.5802
8,2/28/2023,06_01_03,L5,52.5569,51.1396,50.0822,6.40283,10.93460,8.64603,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.4173,-1.0574
9,2/28/2023,06_01_06,L5,49.7725,51.0181,50.9027,2.39693,6.79830,8.20679,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-0.1154


In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [ ]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'slice' in f[-17:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(roi_file)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)


        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from scnn1a_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        
        # open these various files and store them to the pandas dataframe
        # The first three ROIs of each should correspond to the 3 desired barrel ROIs
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        print(new_row)
        df = df.append(new_row, ignore_index=True)




scraping data from  02_01_01.zda using slice2_rois.dat on date: 2/8/2023
{'Date': '2/8/2023', 'Slice/Loc/Rec': '02_01_01', 'Age': 79, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200,150', 'L4 barrel 1 Latency': 50.2296, 'L4 barrel 2 Latency': 50.2176, 'L4 barrel 3 Latency': 53.0153, 'L4 barrel 1 SNR': 5.06973, 'L4 barrel 2 SNR': 7.08936, 'L4 barrel 3 SNR': 3.23049, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\\selected_zda/ROIs_02_01_01.jpg'}


scraping data from  02_01_02.zda using slice2_rois.dat on date: 2/8/2023
{'Date': '2/8/2023', 'Slice/Loc/Rec': '02_01_02', 'Age': 79, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200,150', 'L4 barrel 1 Latency': 49.047, 'L4 barrel 2 Latency': 49.4821, 'L4 barrel 3 Latency': 48.2181, 'L4 barrel 1 SNR': 3.3196, 'L4 barrel 2 SNR': 6.68988, 'L4 barrel 3 SNR': 1.7333, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\\selected_zda/ROIs_02_01_02.jpg'}


scraping data from  02_01_04.zda using slic

In [ ]:
print(df)

In [ ]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

In [ ]:
# write the pandas dataframe back to csv
csv_filename = 'scnn1a_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)